### Import Statements

In [1]:
import random
import json
import pickle
import numpy as np

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yuri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yuri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import tensorflow as tf

## Load training data

In [4]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

In [5]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

In [6]:
for intent in intents['intents']:
    for pattern in intent['pattern']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


print('documents: ', documents)
print('words', words)
print('classes', classes)

documents:  [(['hello'], 'greetings'), (['hey'], 'greetings'), (['hi', 'and', 'hey'], 'greetings'), (['cya'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['have', 'a', 'good', 'day'], 'goodbye')]
words ['hello', 'hey', 'hi', 'and', 'hey', 'cya', 'See', 'you', 'later', 'goodbye', 'have', 'a', 'good', 'day']
classes ['greetings', 'goodbye']


## Prepair training data


In [7]:
# Removes all ignoring letters from the words
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
# Removes duplicates by turning the words list to a set, and reversing it back to a list by using the sorted function
words = sorted(set(words))

print('Words', words)


# Also remove duplicates in classes list
classes = sorted(set(classes))
print('Classes', classes)

Words ['See', 'a', 'and', 'cya', 'day', 'good', 'goodbye', 'have', 'hello', 'hey', 'hi', 'later', 'you']
Classes ['goodbye', 'greetings']


In [8]:
# Save words and classes into pickle files
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

### Machine learning

In [9]:
training = []
output_empty = [0] * len(classes)

In [10]:
for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(output_empty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

In [11]:
random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

## Neural Network

In [12]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

In [13]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [14]:
hist = model.fit(trainX, trainY, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')
print('Done')

Epoch 1/200
2/2 [==============================] - 1s 9ms/step - loss: 0.6959 - accuracy: 0.4286
Epoch 2/200
2/2 [==============================] - 0s 6ms/step - loss: 0.7318 - accuracy: 0.2857
Epoch 3/200
2/2 [==============================] - 0s 8ms/step - loss: 0.6395 - accuracy: 0.7143
Epoch 4/200
2/2 [==============================] - 0s 8ms/step - loss: 0.8017 - accuracy: 0.2857
Epoch 5/200
2/2 [==============================] - 0s 6ms/step - loss: 0.6710 - accuracy: 0.5714
Epoch 6/200
2/2 [==============================] - 0s 5ms/step - loss: 0.6307 - accuracy: 0.7143
Epoch 7/200
2/2 [==============================] - 0s 5ms/step - loss: 0.6095 - accuracy: 0.7143
Epoch 8/200
2/2 [==============================] - 0s 5ms/step - loss: 0.6301 - accuracy: 0.8571
Epoch 9/200
2/2 [==============================] - 0s 5ms/step - loss: 0.5846 - accuracy: 0.8571
Epoch 10/200
2/2 [==============================] - 0s 5ms/step - loss: 0.6167 - accuracy: 0.7143
Epoch 11/200
2/2 [===========

In [15]:
# Save model
model.save('chatbot_model.h5', hist)
print('finished')

finished
